<a href="https://colab.research.google.com/github/jinglescode/python-signal-processing/blob/main/tutorials/Task-Related%20Component%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task-Related Component Analysis

Task-related component analysis (TRCA) is a classifier, originally for steady-state visual evoked potentials (SSVEPs) detection.

Taken from the [paper](http://ieeexplore.ieee.org/document/7904641/) abstract:
> Task-related component analysis (TRCA), which can enhance reproducibility of SSVEPs across multiple trials, was employed to improve the signal-to-noise ratio (SNR) of SSVEP signals by removing background electroencephalographic (EEG) activities. An ensemble method was further developed to integrate TRCA filters corresponding to multiple stimulation frequencies.

In [1]:
#@title 
!git clone https://github.com/jinglescode/python-signal-processing.git
%cd python-signal-processing
!pip install -r requirements.txt --quiet

Cloning into 'python-signal-processing'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 198 (delta 111), reused 118 (delta 50), pack-reused 0
Receiving objects: 100% (198/198), 22.08 MiB | 36.88 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/content/python-signal-processing


In [2]:
from splearn.cross_decomposition.trca import TRCA # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/cross_decomposition/trca.py
from splearn.data.sample_ssvep import SampleSSVEPData # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/data/sample_ssvep.py
from splearn.cross_validate.leave_one_out import leave_one_block_evaluation # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/cross_validate.leave_one_out.py
from splearn.cross_decomposition.cca import CCA # https://github.com/jinglescode/python-signal-processing/blob/main/splearn/cross_decomposition/cca.py

import numpy as np
from sklearn.metrics import accuracy_score

## Load data

In this tutorial, we load a 40-target steady-state visual evoked potentials (SSVEP) dataset recorded from a single subject. It contains 6 blocks, each block consists of 40 trials, where each trial is a target. The electroencephalogram (EEG) signals has 9 channels and 1250 sampling points.

Read more about this dataset: https://www.pnas.org/content/early/2015/10/14/1508080112.abstract.

In [3]:
data = SampleSSVEPData()
eeg = data.get_data()
labels = data.get_targets()
print("eeg.shape:", eeg.shape)
print("labels.shape:", labels.shape)

eeg.shape: (6, 40, 9, 1250)
labels.shape: (6, 40)


## Leave-One-Block-Out cross-validation

We use the Leave-One-Block-Out cross-validation approach to determine TRCA's classification performance.

In [4]:
trca_classifier = TRCA(sampling_rate=data.sampling_rate)
test_accuracies = leave_one_block_evaluation(classifier=trca_classifier, X=eeg, Y=labels)

Block: 1 | Train acc: 100.00% | Test acc: 97.50%
Block: 2 | Train acc: 100.00% | Test acc: 100.00%
Block: 3 | Train acc: 100.00% | Test acc: 100.00%
Block: 4 | Train acc: 100.00% | Test acc: 100.00%
Block: 5 | Train acc: 100.00% | Test acc: 97.50%
Block: 6 | Train acc: 100.00% | Test acc: 100.00%
Mean test accuracy: 99.2%


### Comparing to CCA
Let's also test the classification performance with [CCA](https://colab.research.google.com/github/jinglescode/python-signal-processing/blob/main/tutorials/Canonical%20Correlation%20Analysis.ipynb) and compare the accuracy performance.

In [5]:
cca = CCA(
    sampling_rate=data.sampling_rate, 
    target_frequencies=data.get_stimulus_frequencies(), 
    signal_size=eeg.shape[3], 
    num_harmonics=1
)

test_accuracies = leave_one_block_evaluation(classifier=cca, X=eeg, Y=labels)

Block: 1 | Train acc: 100.00% | Test acc: 100.00%
Block: 2 | Train acc: 100.00% | Test acc: 100.00%
Block: 3 | Train acc: 100.00% | Test acc: 100.00%
Block: 4 | Train acc: 100.00% | Test acc: 100.00%
Block: 5 | Train acc: 100.00% | Test acc: 100.00%
Block: 6 | Train acc: 100.00% | Test acc: 100.00%
Mean test accuracy: 100.0%


Comparing the `mean test accuracy`, we can't see the difference in the classification performance between TRCA and CCA. We will use another dataset below.

## Using `.fit` and `.predict`

In this example, we select the first 2 blocks for training and the remaining 4 blocks for testing. 

In [6]:
trca_classifier = TRCA(sampling_rate=data.sampling_rate)

x_train = eeg[0:2]
y_train = labels[0:2]

blocks, targets, channels, samples = x_train.shape
x_train = x_train.reshape((blocks-1*targets, channels, samples))
y_train = y_train.reshape((blocks-1*targets))

print("Train shape:", x_train.shape, y_train.shape)
trca_classifier.fit(x_train, y_train)

for block_i in range(2, 6):

    test_x = eeg[block_i]
    test_y = labels[block_i]

    # Shuffle the test set
    arrangement = np.arange(40)
    np.random.shuffle(arrangement)
    test_x = test_x[arrangement, :,:]
    test_y = test_y[arrangement]

    # Preduct
    pred = trca_classifier.predict(test_x)
    acc = accuracy_score(test_y, pred)

    print(f'Block: {block_i+1} | accuracy: {acc*100:.2f}%')

Train shape: (80, 9, 1250) (80,)
Block: 3 | accuracy: 100.00%
Block: 4 | accuracy: 100.00%
Block: 5 | accuracy: 97.50%
Block: 6 | accuracy: 100.00%


## Another dataset, HS-SSVEP

As we can't see the difference in classification performance with the previous data, in this example we will evaluate with a single subject data taken from the [Tsinghua SSVEP benchmark dataset](https://ieeexplore.ieee.org/document/7740878).

In the following code blocks, we will download and prepare the data and labels.

In [7]:
!wget -r --no-parent ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S33.mat

--2021-08-18 13:12:13--  ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S33.mat
           => ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’
Resolving sccn.ucsd.edu (sccn.ucsd.edu)... 169.228.38.2
Connecting to sccn.ucsd.edu (sccn.ucsd.edu)|169.228.38.2|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/ssvep_benchmark_dataset ... done.
==> PASV ... done.    ==> LIST ... done.

sccn.ucsd.edu/pub/s     [ <=>                ]   2.78K  --.-KB/s    in 0s      

2021-08-18 13:12:14 (261 MB/s) - ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’ saved [2850]

Removed ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/.listing’.
--2021-08-18 13:12:14--  ftp://anonymous@sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S33.mat
           => ‘sccn.ucsd.edu/pub/ssvep_benchmark_dataset/S33.mat’
==> CWD not required.
==> PASV ... done.    ==> RETR S33.mat ... done.
Length: 106223727 (101M)

sccn.ucsd.edu/pub/s 

In [8]:
from scipy.io import loadmat

# select channels
ch_names = ['FP1','FPZ','FP2','AF3','AF4','F7','F5','F3','F1','FZ','F2','F4','F6','F8','FT7','FC5','FC3','FC1','FCz','FC2','FC4','FC6','FT8','T7','C5','C3','C1','Cz','C2','C4','C6','T8','M1','TP7','CP5','CP3','CP1','CPZ','CP2','CP4','CP6','TP8','M2','P7','P5','P3','P1','PZ','P2','P4','P6','P8','PO7','PO5','PO3','POz','PO4','PO6','PO8','CB1','O1','Oz','O2','CB2']
ch_index = [47,53,54,55,56,57,60,61,62]

sampling_rate = 250

folder = 'sccn.ucsd.edu/pub/ssvep_benchmark_dataset'
data = loadmat(f"{folder}/S33.mat")
eeg = data['data']
eeg = eeg.transpose((3, 2, 0, 1))
eeg = eeg[:,  :, ch_index, 250:500]
print("Data shape:", eeg.shape)

blocks, targets, channels, samples = eeg.shape
y_train = np.tile(np.arange(0, targets), (1, blocks-1)).squeeze()
y_test = np.arange(0, targets)
print("Label shape:", y_train.shape, y_test.shape)

Data shape: (6, 40, 9, 250)
Label shape: (200,) (40,)


## Classification with TRCA

In [9]:
trca_classifier = TRCA(sampling_rate=sampling_rate)
test_accuracies = leave_one_block_evaluation(classifier=trca_classifier, X=eeg, Y=labels)

Block: 1 | Train acc: 100.00% | Test acc: 70.00%
Block: 2 | Train acc: 100.00% | Test acc: 47.50%
Block: 3 | Train acc: 100.00% | Test acc: 67.50%
Block: 4 | Train acc: 100.00% | Test acc: 47.50%
Block: 5 | Train acc: 100.00% | Test acc: 70.00%
Block: 6 | Train acc: 100.00% | Test acc: 62.50%
Mean test accuracy: 60.8%


### Comparing to CCA

In [10]:
stimulus_frequencies = np.array([8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,8.2,9.2,10.2,11.2,12.2,13.2,14.2,15.2,8.4,9.4,10.4,11.4,12.4,13.4,14.4,15.4,8.6,9.6,10.6,11.6,12.6,13.6,14.6,15.6,8.8,9.8,10.8,11.8,12.8,13.8,14.8,15.8])

cca = CCA(
    sampling_rate=sampling_rate, 
    target_frequencies=stimulus_frequencies,
    signal_size=eeg.shape[3], 
    num_harmonics=2
)

test_accuracies = leave_one_block_evaluation(classifier=cca, X=eeg, Y=labels)

Block: 1 | Train acc: 20.50% | Test acc: 35.00%
Block: 2 | Train acc: 26.00% | Test acc: 7.50%
Block: 3 | Train acc: 22.00% | Test acc: 27.50%
Block: 4 | Train acc: 23.50% | Test acc: 20.00%
Block: 5 | Train acc: 23.50% | Test acc: 20.00%
Block: 6 | Train acc: 22.00% | Test acc: 27.50%
Mean test accuracy: 22.900000000000002%
